<a href="https://colab.research.google.com/github/singaravelan/TensorFlow_programming/blob/main/1.%20AI%20Power%20ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import pandas as pd

In [ ]:
import os
import zipfile
import wget

# Specify the URL and destination folder
download_url = 'https://www.dropbox.com/scl/fi/mnmxwpgyqz2c6uib960th/dialogs.txt?rlkey=y8p0zm33sj9iqopmihqx8jabs&raw=1'
destination_folder = 'data'

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Download the file using wget
zip_file_path = os.path.join(destination_folder, 'dialog.txt')
wget.download(download_url, zip_file_path)

'data/dialog (1).txt'

In [ ]:
# Load dialogues from a text file
file_path = '/Users/singaravelang/Documents/Learning/Workshop/data/dialog.txt'  # Change this to the path of your text file

df = pd.read_csv(file_path,sep='\t')

df.columns = ["Question","Answer"]

In [ ]:
# Separate input and target responses
inputs, targets = df["Question"].tolist(), df["Answer"].tolist()

# Tokenize input and target sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(inputs + targets)

input_sequences = tokenizer.texts_to_sequences(inputs)
target_sequences = tokenizer.texts_to_sequences(targets)

input_sequences_padded = pad_sequences(input_sequences, padding='post')
target_sequences_padded = pad_sequences(target_sequences, padding='post')



In [ ]:
# Build the Seq2Seq model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=input_sequences_padded.shape[1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.RepeatVector(target_sequences_padded.shape[1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Dense(len(tokenizer.word_index) + 1, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)

# Train the model
model.fit(input_sequences_padded, target_sequences_padded, epochs=500)

Epoch 1/500
117/117 [==============================] - 7s 25ms/step - loss: 3.3517 - accuracy: 0.6502
Epoch 2/500
117/117 [==============================] - 3s 24ms/step - loss: 2.3948 - accuracy: 0.6574
Epoch 3/500
117/117 [==============================] - 3s 25ms/step - loss: 2.3584 - accuracy: 0.6646
Epoch 4/500
117/117 [==============================] - 3s 26ms/step - loss: 2.3385 - accuracy: 0.6650
Epoch 5/500
117/117 [==============================] - 3s 26ms/step - loss: 2.3280 - accuracy: 0.6649
Epoch 6/500
117/117 [==============================] - 3s 26ms/step - loss: 2.3120 - accuracy: 0.6652
Epoch 7/500
117/117 [==============================] - 3s 25ms/step - loss: 2.3005 - accuracy: 0.6657
Epoch 8/500
117/117 [==============================] - 3s 25ms/step - loss: 2.2909 - accuracy: 0.6656
Epoch 9/500
117/117 [==============================] - 3s 25ms/step - loss: 2.2790 - accuracy: 0.6661
Epoch 10/500
117/117 [==============================] - 3s 25ms/step - loss: 2.268

In [ ]:
# Inference function
def chatbot_response(input_text):
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_seq_padded = pad_sequences(input_seq, padding='post', maxlen=input_sequences_padded.shape[1])
    predicted_probabilities = model.predict(input_seq_padded)
    predicted_seq = np.argmax(predicted_probabilities, axis=-1)
    predicted_text = tokenizer.sequences_to_texts(predicted_seq)[0]
    return predicted_text

# Test the chatbot
user_input = "Hi"
while user_input.lower() != 'bye':
    response = chatbot_response(user_input)
    print(f"User: {user_input}")
    print(f"Chatbot: {response}")
    user_input = input("User: ")

1/1 [==============================] - 1s 948ms/step
User: Hi
Chatbot: bye time
1/1 [==============================] - 0s 14ms/step
User: hi how are you
Chatbot: what that
1/1 [==============================] - 0s 14ms/step
User: how was your day
Chatbot: what it it it
1/1 [==============================] - 0s 14ms/step
User: 
Chatbot: i are a the
